In [1]:
import numpy as np
import pandas as pd

import scipy.stats as sts
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv('tracks_edited.csv', sep=',')

In [5]:
df

,artist_name,song_name,duration,playcount,listeners,tag,name_size,playcounts_per_listener
0,SZA,Kill Bill,2.566667,17429320,987007,pop,9,17.658760
1,"Tyler, the Creator",See You Again (feat. Kali Uchis),3.000000,16466512,1002965,rap,32,16.417833
2,pinkpantheress,Boy's a liar Pt. 2,3.166667,8772363,724864,rap,18,12.102081
3,Frank Ocean,Pink + White,3.066667,21272327,1105199,rnb,12,19.247508
4,Fifty Fifty,Cupid - Twin Ver.,3.166667,3895874,355420,pop,17,10.961325
...,...,...,...,...,...,...,...,...
1177,Ariana Grande,pov,3.333333,7396618,472265,rnb,3,15.662008
1178,Nirvana,All Apologies,4.016667,9185567,1364470,Grunge,13,6.731967
1179,NMixx,DICE,3.166667,4006601,177227,banger cry about it,4,22.607170
1180,SZA,Doves in the Wind (feat. Kendrick Lamar),4.433333,3815810,387544,alternative rnb,40,9.846134


### 1. Гипотеза о том, что мат ожидание playcount тэга pop больше, чем мат ожидание прослушиваний всех остальных жанров 

In [18]:
from scipy.stats import ttest_ind

In [19]:
ttest_ind(df[df.tag=='pop'].playcount, df[df.tag!='pop'].playcount, alternative='less')

Ttest_indResult(statistic=3.5619191233717964, pvalue=0.9998085868598864)

In [20]:
ttest_ind(df[df.tag=='pop'].playcount, df[df.tag!='pop'].playcount, alternative='less')[1]>=0.05

True

Мы проверяем нулевую гипотезу о том, что мат ожидание playcount тэга pop больше, чем мат ожидание прослушиваний всех остальных жанров при уровне значимости альфа 0.05: p-value>alpha, поэтому нет оснований отвергать нулевую гипотезу. Мы выбрали именно двухвыборочный ttest (односторонний), так как мы не знаем истинные дисперсии выборок из истинных распределений. Получается, что полученный результат не проиворечит данным визуализации.

### 2. Гипотеза о том, что мат ожидания k-pop и folk тэгов равны

In [21]:
ttest_ind(df[df.tag=='k-pop'].playcount, df[df.tag=='folk'].playcount)

Ttest_indResult(statistic=-1.401878850424946, pvalue=0.16628060947685228)

In [23]:
ttest_ind(df[df.tag=='k-pop'].playcount, df[df.tag=='folk'].playcount)[1]>=0.05

True

Мы проверяем нулевую гипотезу о том, что мат ожиданиz k-pop и folk тэгов равны при уровне значимости альфа 0.05: p-value>alpha, поэтому нет оснований отвергать нулевую гипотезу. Мы выбрали именно двухвыборочный ttest (двухсторонний), так как мы не знаем истинные дисперсии выборок из истинных распределений. Получается, что полученный результат не проиворечит данным визуализации.

### 2. Гипотеза о том, что дисперсии k-pop и folk тэгов равны

In [24]:
len(df[df.tag=='k-pop'].playcount), len(df[df.tag=='folk'].playcount)

(33, 27)

In [25]:
var_kpop=(np.std(df[df.tag=='k-pop'].playcount))**2

In [26]:
var_folk=(np.std(df[df.tag=='folk'].playcount))**2

In [27]:
f_stat=var_kpop/var_folk

In [28]:
from scipy.stats import f

In [29]:
F_crit = f.ppf(q=0.05, dfn=33-1, dfd=27-1)

In [32]:
f_stat<=F_crit

False

Проверяем нулевую гипотезу о равенстве дисперсий с помощью теста Фишера. Рассчитали вручную ф-статистику, нашли квантиль для альфа 0.05 для распределения с конкретными степенями свободы. Оказалось, что наблюдаемая статистика (f_stat) лежит правее критического значения, то есть попадает в область отвержения нулевой гипотезы. Следовательно, нулевая гипотеза отвергается. 

# Вывод

В итоге: 
Согласно результатам нашего исследования, песни с тэгом pop наиболее популярны, так как имеют наибольшее количество прослушиваний на данной платформе. 
Также, тесты по данным тэгов k-pop и folk действительно не дали оснований отвергать гипотезу о том, что мат ожидания прослушиваний равны, но гипотеза о равенстве дисперсий была отвергнута, следовательно, данные выборки не имеют одинаковых истинных распределений, хотя результаты визуализаций демонстрируют их схожесть. 